Sources to ingest data from Confluent

In [ ]:
CREATE SOURCE cam1 (frame BYTEA)
INCLUDE timestamp AS ts
WITH (
    connector='kafka',
    topic='cam1',
    properties.bootstrap.server='pkc-ldvr1.asia-southeast1.gcp.confluent.cloud:9092',
    properties.security.protocol = 'SASL_SSL',
    properties.sasl.mechanism = 'PLAIN',
    properties.sasl.username = 'SP3DSAA5OXMFOKI2',
    properties.sasl.password = 'LFGeoUX2nhyrdTCGlxwP8kdNXtRVXM2Xz2dR1VsvowM3iR61Qv5bUL2xdBtFeZnh'
) FORMAT PLAIN ENCODE BYTES;

CREATE SOURCE cam2 (frame BYTEA)
INCLUDE timestamp AS ts
WITH (
    connector='kafka',
    topic='cam2',
    properties.bootstrap.server='pkc-ldvr1.asia-southeast1.gcp.confluent.cloud:9092',
    properties.security.protocol = 'SASL_SSL',
    properties.sasl.mechanism = 'PLAIN',
    properties.sasl.username = '',
    properties.sasl.password = ''
) FORMAT PLAIN ENCODE BYTES;

Create UDF

In [ ]:
CREATE FUNCTION read_plates(BYTEA, TIMESTAMP) RETURNS TABLE(plate VARCHAR, ts TIMESTAMP)
AS 'read_plates' USING LINK 'http://34.48.71.202:8815';

Create MV to store result of UDF

In [ ]:
CREATE MATERIALIZED VIEW cam1_struct AS (
  SELECT read_plates(frame, ts AT TIME ZONE 'Asia/Singapore')
  FROM cam1
);

CREATE MATERIALIZED VIEW cam2_struct AS (
  SELECT read_plates(frame, ts AT TIME ZONE 'Asia/Singapore')
  FROM cam2
);

Unpack data from structs into columns

In [ ]:
CREATE MATERIALIZED VIEW cam1_mv AS (
  SELECT (read_plates).plate,
  (read_plates).ts
  FROM cam1_struct
);

CREATE MATERIALIZED VIEW cam2_mv AS (
  SELECT (read_plates).plate,
  (read_plates).ts
  FROM cam2_struct
);

Common plates between the two cameras

In [ ]:
CREATE MATERIALIZED VIEW common AS (
  SELECT DISTINCT ON (cam1_mv.plate)
    cam1_mv.plate,
    cam1_mv.ts AS cam1_ts,
    cam2_mv.ts AS cam2_ts
  FROM cam1_mv, cam2_mv
  WHERE cam1_mv.plate = cam2_mv.plate
);

Number of distinct cars in the last x minutes

In [ ]:
CREATE MATERIALIZED VIEW distinct_cars AS (
  SELECT COUNT(DISTINCT plate) AS distinct_vehicle_count,
  window_start,
  window_end
  FROM TUMBLE(cam1_mv, ts, INTERVAL '2 minutes')
  GROUP BY window_start, window_end
);

Average interval between cameras

In [ ]:
CREATE MATERIALIZED VIEW avg_time_interval AS (
  SELECT AVG(elapsed_time) AS average_interval FROM (
    SELECT extract(second from cam2_ts-cam1_ts) AS elapsed_time
    FROM common
  )
);

Number of cars each minute

In [ ]:
CREATE MATERIALIZED VIEW cars_per_min AS (
  SELECT COUNT(*) AS vehicle_count,
  window_start AS start_time,
  window_end AS end_time
  FROM TUMBLE(cam1_mv, ts, INTERVAL '1 minutes')
  GROUP BY window_start, window_end
  ORDER BY window_start
);



Average cars per minute in each 5 minute interval

In [ ]:
CREATE MATERIALIZED VIEW avg_cars AS (
  SELECT AVG(vehicle_count) AS avg_vehicles_per_minute,
  window_start,
  window_end
  FROM TUMBLE(cars_per_min, start_time, INTERVAL '5 minutes')
  GROUP BY window_start, window_end
);


Top 5 times with most cars

In [ ]:
CREATE MATERIALIZED VIEW top_5_times AS (
  SELECT *
  FROM cars_per_min
  ORDER BY vehicle_count DESC
  LIMIT 5
);